In [ ]:
import cv2
import numpy as np

In [ ]:
classesFile = 'coco.names'
file = open(classesFile, 'r')
classesNames = file.read().split('\n')

print('No. of Classes: ', len(classesNames), '\n')
print(classesNames)

In [ ]:
cam = cv2.VideoCapture(0)

modelConfig = 'yolov3-320.cfg'
modelWeights = 'yolov3-320.weights'

model = cv2.dnn.readNetFromDarknet(modelConfig, modelWeights)
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
def findObject(Outputs, Img):
    hT, wT, cT = Img.shape
    
    bbox = []
    classIds = []
    confs = []
    
    for output in Outputs:
        for det in output:
            score = det[5:]
            classId = np.argmax(score)
            confidence = score[classId]
            if confidence > 0.5:
                w, h = int(det[2]*wT), int(det[2]*hT)
                x, y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
#     print(len(bbox))
#     print(bbox)
    indices = cv2.dnn.NMSBoxes(bbox, confs, 0.5, 0.3)
    for i in indices:
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 255), 2)
        cv2.putText(img, f'{classesNames[classIds[i]].upper() } {int(confs[i]*100)}%', 
                   (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        
    cv2.imwrite('Det.jpg', img)

In [ ]:
while True:
    success, img = cam.read()
    
    blob = cv2.dnn.blobFromImage(img, 1/255, (320, 320), [0,0,0], 1, crop=False)
    model.setInput(blob)
    
    LayersName = model.getLayerNames()
    outputLayer = [LayersName[i-1] for i in model.getUnconnectedOutLayers()]
    
    output = model.forward(outputLayer)
    findObject(output, img)
    
#     print(len(output))
#     for i  in output:
#         print(i.shape)

    cv2.imshow('Image', img)
    cv2.waitKey(1)